# Functions

In [ ]:
def add_missing_PID_df(orig_df, model_df):
  unique_PID_cases = orig_df.PID.unique()
  out_df = model_df.copy()
  for PID_current in unique_PID_cases:
    if PID_current not in model_df.PID.unique():
      #retrieve metadata from orig_df
      orig_caseID_serie = orig_df[orig_df["PID"] == PID_current].head(1).squeeze()
      #overwrite values with volume=0mL
      orig_caseID_serie["ai_Value"] = 0
      orig_caseID_serie["SegmentAlgorithmName"] = model_df.SegmentAlgorithmName.unique()[0]
      #add to current model_df
      out_df = pd.concat([out_df, pd.DataFrame([orig_caseID_serie])],
                           ignore_index=True)
  return out_df

In [ ]:
def calculate_time_index(row, df_input, studyDateColName="image_StudyDate"): #assumes only two studies per PatientID
  other_study_time = df_input[(df_input["image_PatientID"] == row["image_PatientID"])
    & (df_input["StudyInstanceUID"] != row["StudyInstanceUID"])][studyDateColName].values[0]
  if row[studyDateColName] < other_study_time:
    return "T0"
  elif row[studyDateColName] > other_study_time:
    return "T1"
  else:
    return "T"

In [ ]:
def calculate_time_index_prostatex_inf_only(row, df_input,
                         seriesTimecolName="image_SeriesTime"): #assumes only two studies per PatientID
  seriesTimesLst = df_input[(df_input["image_PatientID"] == row["image_PatientID"])
    & (df_input["StudyInstanceUID"] == row["StudyInstanceUID"])][seriesTimecolName].unique()
  for idx, el in enumerate(sorted(seriesTimesLst)):
    if el == row[seriesTimecolName]:
      return f"T{idx}"

# Imports

In [ ]:
import os
import numpy as np
import glob
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
# import colorcet as cc
import os
import plotly.io as pio

In [ ]:
# #colab
# from google.colab import auth
# auth.authenticate_user()
# project_id = "idc-sandbox-003"
# os.environ["GCP_PROJECT_ID"] = project_id
# from google.cloud import bigquery
# bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Data import

In [ ]:
# selection_query = f"""
# SELECT * EXCEPT(asd)
# FROM `idc-sandbox-003.prostate_seg_terra_mhub_v3.final_table_looker_studio`
#  """
# selection_result = bq_client.query(selection_query)
# data_new_df = selection_result.result().to_dataframe()
# data_new_df["algorithmNameCollection"] = data_new_df.apply(lambda x : x["SegmentAlgorithmName"]+"-"+x["collection_id"], axis=1)
# data_new_df.to_csv("analysis_results.csv", index=False)

In [ ]:
!wget https://raw.githubusercontent.com/ImagingDataCommons/idc-prostate-mri-analysis/refs/heads/main/analysis_results/analysis_results.csv -O analysis_results.csv
data_new_df = pd.read_csv("analysis_results.csv")

In [ ]:
# casses_to_exclude = ["ProstateX-0054", "ProstateX-0148", "ProstateX-0260", "ProstateX-0272", "ProstateX-0052"]
# selection_query = f"""
# SELECT
#   image_PatientID,
#   image_StudyDate,
#   image_SeriesDate,
#   image_StudyTime,
#   image_SeriesTime,
#   sourceSegmentedSeriesUID,
#   image_SeriesInstanceUID,
#   StudyInstanceUID,
#   collection_id,
#   SegmentAlgorithmName[SAFE_OFFSET(0)] AS SegmentAlgorithmName,
#   Value AS ai_Value,
#   CONCAT('https://fir-idc-prostate-ohif.web.app/viewer/', StudyInstanceUID, '!secondGoogleServer=/projects/idc-sandbox-003/locations/us/datasets/prostate_seg_terra_prostatex_inf_only_mhub_dataset/dicomStores/dicomAllAIIdcSegSr') as ohif_url,
#   SegmentedPropertyType.CodeMeaning AS SegmentedPropertyTypeCodeMeaning,
#   SegmentedPropertyType.CodeValue AS SegmentedPropertyTypeCodeValue,
#   SegmentedPropertyType.CodingSchemeDesignator AS SegmentedPropertyTypeCodingSchemeDesignator
# FROM
#   `idc-sandbox-003.prostate_seg_terra_mhub_v3_prostatex_inf_only.ai_processed_seg_sr_eval_results_table`
# WHERE
#   Quantity.CodeMeaning = 'Volume of Mesh'
#  """
# selection_result = bq_client.query(selection_query)
# data_prostatex_inf_only_df = selection_result.result().to_dataframe()
# data_prostatex_inf_only_df = data_prostatex_inf_only_df[~data_prostatex_inf_only_df.image_PatientID.isin(casses_to_exclude)]
# # data_prostatex_inf_only_df["algorithmNameCollection"] = data_prostatex_inf_only_df.apply(lambda x : x["SegmentAlgorithmName"]+"-"+x["collection_id"], axis=1)

In [ ]:
# data_prostatex_inf_only_df.to_csv("analysis_results_prostatex_inf_only.csv", index=False)
!wget https://raw.githubusercontent.com/ImagingDataCommons/idc-prostate-mri-analysis/refs/heads/main/analysis_results/analysis_results_prostatex_inf_only.csv -O analysis_results_prostatex_inf_only.csv
data_prostatex_inf_only_df = pd.read_csv("analysis_results_prostatex_inf_only.csv")

In [ ]:
data_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4909 entries, 0 to 4908
Data columns (total 61 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   segmentationInstanceUID               4907 non-null   object 
 1   sourceSegmentedSeriesUID              4907 non-null   object 
 2   expert_QuantityCodeMeaning            4909 non-null   object 
 3   expert_QuantityCodeValue              4909 non-null   object 
 4   expert_Value                          4909 non-null   object 
 5   expert_UnitsCodeValue                 4909 non-null   object 
 6   expert_UnitsCodeMeaning               4909 non-null   object 
 7   expert_findingSiteCodeValue           4909 non-null   object 
 8   expert_findingSiteCodeMeaning         4909 non-null   object 
 9   ai_QuantityCodeMeaning                4907 non-null   object 
 10  ai_QuantityCodeValue                  4907 non-null   object 
 11  ai_QuantityCoding

# Figures

### Whole prostate quantitative analysis

In [ ]:
# array(['prostate158-prostate_mri_us_biopsy-prostate',
#        'nnunet_task024-prostate_mri_us_biopsy-prostate',
#        'prostate158-qin_prostate_repeatability-prostate',
#        'nnunet_task05-qin_prostate_repeatability-prostate',
#        'nnunet_task024-qin_prostate_repeatability-prostate',
#        'prostate158-prostatex-zonal', 'prostate158-prostatex-prostate',
#        'nnunet_task05-prostatex-prostate',
#        'nnunet_task05-prostatex-zonal',
#        'nnunet_task024-prostatex-prostate'], dtype=object)

In [ ]:
data_new_df.algorithmNameCollection.unique()

array(['nnunet_prostate_zonal_task05-prostatex',
       'monai_prostate158-prostatex', 'bamf_nnunet_mr_prostate-prostatex',
       'nnunet_prostate_task24-prostatex',
       'bamf_nnunet_mr_prostate-prostate_mri_us_biopsy',
       'nnunet_prostate_task24-prostate_mri_us_biopsy',
       'nnunet_prostate_zonal_task05-prostate_mri_us_biopsy',
       'monai_prostate158-prostate_mri_us_biopsy',
       'bamf_nnunet_mr_prostate-qin_prostate_repeatability',
       'nnunet_prostate_zonal_task05-qin_prostate_repeatability',
       'monai_prostate158-qin_prostate_repeatability',
       'nnunet_prostate_task24-qin_prostate_repeatability'], dtype=object)

In [ ]:
data_new_df.SegmentedPropertyTypeCodeMeaning.unique()

array(['Prostate', 'Prostatic structure',
       'Structure of peripheral glandular zone of prostate (body structure)',
       'Structure of transition zone of prostate (body structure)'],
      dtype=object)

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# fig, ax = plt.subplots() # define the axis object here
# plt.style.use('fivethirtyeight')
# %matplotlib inline
# # sns.reset_defaults()
# plt.rcParams.update(plt.rcParamsDefault)              # <---- set markersize here
# # plt.style.use('default')
# # plt.rcParams.update({'font.size': 0.5})
# plt.style.use('default')
# sns.set(rc={'figure.dpi': 2000})
# plt.style.use('default')

# Setup
plt.style.use('default')
plt.rcParams['figure.dpi'] = 1000

# Create subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# # fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
# plt.rcParams['figure.dpi'] = 2000
# plt.style.use('default')
temp_df = data_new_df[data_new_df.SegmentedPropertyTypeCodeMeaning.isin(['Prostate', 'Prostatic structure'])]
temp_df["algorithmNameCollection"] = temp_df.algorithmNameCollection.apply(lambda x : "-".join(x.split('-')[0:1]))
# temp_df = temp_df[temp_df.asd >= 0]#eliminate outliers set to -999
# sns.set_style('white'
a = sns.boxplot(data=temp_df, x="collection_id",
                     y="dsc",
                     hue="algorithmNameCollection", ax=ax1,
                     dodge=True, width=0.3, fliersize=0.7, fill=False)
b = sns.boxplot(data=temp_df, x="collection_id",
                     y="hsdff",
                     hue="algorithmNameCollection", ax=ax2,
                     dodge=True, width=0.3, fliersize=0.7,  fill=False)
a.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
b.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
# c.legend(loc='upper right', bbox_to_anchor=(1.25, 0.6))
ax1.tick_params(axis='both',labelsize=15)
ax1.set_ylabel('DSC', fontsize = 20.0) # Y label
ax1.set_xlabel('Collections', fontsize = 20) # X label
a.set(xlabel=None)
ax2.tick_params(axis='both',labelsize=15)
ax2.set_ylabel('HD', fontsize = 20.0) # Y label
ax2.set_xlabel('Collections', fontsize = 20) # X label
plt.show()

### Peripheral zone quantitative analysis

In [ ]:
# Setup
plt.style.use('default')
plt.rcParams['figure.dpi'] = 1000

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

temp_df = data_new_df[data_new_df.SegmentedPropertyTypeCodeMeaning == 'Structure of peripheral glandular zone of prostate (body structure)']
temp_df["algorithmNameCollection"] = temp_df.algorithmNameCollection.apply(lambda x : "-".join(x.split('-')[0:1]))
# sns.set_style('white')
plt.rcParams.update({'font.size': 13})
# plt.rc('xtick', labelsize=20)
# plt.rc('ytick', labelsize=20)
plt.xlabel('xlabel', fontsize=18)
plt.ylabel('ylabel', fontsize=16)
a = sns.boxplot(data=temp_df, x="collection_id",
                     y="dsc",
                     hue="algorithmNameCollection", ax=ax1,
                     dodge=True, width=0.3, fill=False)
# plt.tick_params(axis='both', which='major', labelsize=22)
b = sns.boxplot(data=temp_df, x="collection_id",
                     y="hsdff",
                     hue="algorithmNameCollection", ax=ax2,
                     dodge=True, width=0.3, fill=False)
ax1.tick_params(axis='both',labelsize=15)
ax1.set_ylabel('DSC', fontsize = 20.0) # Y label
ax1.set_xlabel('Collections', fontsize = 20) # X label
ax2.tick_params(axis='both',labelsize=15)
ax2.set_ylabel('HD', fontsize = 20.0) # Y label
ax2.set_xlabel('Collections', fontsize = 20) # X label
ax1.get_legend().remove()
b.legend(loc='upper right', bbox_to_anchor=(1.67, 1.02))
plt.show()

### Transition zone quantitative analysis

In [ ]:
# Setup
plt.style.use('default')
plt.rcParams['figure.dpi'] = 1000

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
plt.style.use('default')
temp_df = data_new_df[data_new_df.SegmentedPropertyTypeCodeMeaning == 'Structure of transition zone of prostate (body structure)']
temp_df["algorithmNameCollection"] = temp_df.algorithmNameCollection.apply(lambda x : "-".join(x.split('-')[0:1]))
# sns.set_style('white')
plt.rcParams.update({'font.size': 13})
# plt.rc('xtick', labelsize=20)
# plt.rc('ytick', labelsize=20)
plt.xlabel('xlabel', fontsize=18)
plt.ylabel('ylabel', fontsize=16)
a = sns.boxplot(data=temp_df, x="collection_id",
                     y="dsc",
                     hue="algorithmNameCollection", ax=ax1,
                     dodge=True, width=0.3, fill=False)
# plt.tick_params(axis='both', which='major', labelsize=22)
b = sns.boxplot(data=temp_df, x="collection_id",
                     y="hsdff",
                     hue="algorithmNameCollection", ax=ax2,
                     dodge=True, width=0.3, fill=False)
ax1.tick_params(axis='both',labelsize=15)
ax1.set_ylabel('DSC', fontsize = 20.0) # Y label
ax1.set_xlabel('Collections', fontsize = 20) # X label
ax2.tick_params(axis='both',labelsize=15)
ax2.set_ylabel('HD', fontsize = 20.0) # Y label
ax2.set_xlabel('Collections', fontsize = 20) # X label
ax1.get_legend().remove()
# ax3.get_legend().remove()
# plt.rcParams.update({'font.size': 22})
b.legend(loc='upper right', bbox_to_anchor=(1.67, 1.02))

plt.show()

## Repeatability analysis

### Repeatability analysis for QIN-Prostate-Repeatability

In [ ]:
temp_df = data_new_df[(data_new_df.SegmentedPropertyTypeCodeMeaning.isin(['Prostate', 'Prostatic structure']))
                  & (data_new_df.collection_id == 'qin_prostate_repeatability')]

In [ ]:
temp_df.algorithmNameCollection.unique()

array(['bamf_nnunet_mr_prostate-qin_prostate_repeatability',
       'nnunet_prostate_zonal_task05-qin_prostate_repeatability',
       'monai_prostate158-qin_prostate_repeatability',
       'nnunet_prostate_task24-qin_prostate_repeatability'], dtype=object)

Whole prostate task024

In [ ]:
temp_prostate_qin_task024_df = temp_df[temp_df.algorithmNameCollection == 'nnunet_prostate_task24-qin_prostate_repeatability']

In [ ]:
temp_prostate_qin_task024_df['time_index'] = temp_prostate_qin_task024_df.apply(calculate_time_index,
                                    args = (temp_prostate_qin_task024_df,), axis=1)
temp_prostate_qin_task024_df["PID"] = temp_prostate_qin_task024_df.apply(lambda x: "PID_"\
                                                               +x["image_PatientID"].split('-')[1]\
                             +"_"+x["time_index"], axis=1)

In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#8b0000']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_task024_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : "        "+x.split("_")[0][-2:]).values)
x_ticks_formatted = [x if idx % 2 == 0 else "" for idx, x in enumerate(x_ticks_formatted)]
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
            #  hover_data=["PatientID"],
              markers=True,
              width=1100, height=400)
# initializing showlegend to "False"
fig_ai.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai.update_xaxes(tickangle=0, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai['data']):
  # if idx % 2 == 0 :
  trace["name"] = "nnunet-task024"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai.show()

Whole prostate task05

In [ ]:
temp_prostate_qin_task025_df = temp_df[(temp_df.algorithmNameCollection == 'nnunet_prostate_zonal_task05-qin_prostate_repeatability') \
                                       & (temp_df.SegmentedPropertyTypeCodeMeaning == 'Prostate')]

In [ ]:
len(temp_prostate_qin_task025_df)

30

In [ ]:
temp_prostate_qin_task025_df['time_index'] = temp_prostate_qin_task025_df.apply(calculate_time_index,
                                    args = (temp_prostate_qin_task025_df,), axis=1)
temp_prostate_qin_task025_df["PID"] = temp_prostate_qin_task025_df.apply(lambda x: "PID_"\
                                                               +x["image_PatientID"].split('-')[1]\
                             +"_"+x["time_index"], axis=1)

In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#0b6623']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_task025_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : "        "+x.split("_")[0][-2:]).values)
x_ticks_formatted = [x if idx % 2 == 0 else "" for idx, x in enumerate(x_ticks_formatted)]
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai2 = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
              markers=True,
              width=1100, height=400)
# initializing showlegend to "False"
fig_ai2.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai2.update_xaxes(tickangle=0, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai2.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai2['data']):
  # if idx % 2 == 0 :
  trace["name"] = "nnunet-task05"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai2['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai2.show()

Whole prostate prostate158

In [ ]:
temp_prostate_qin_monai_df = temp_df[(temp_df.algorithmNameCollection == 'monai_prostate158-qin_prostate_repeatability') \
                                     & (temp_df.SegmentedPropertyTypeCodeMeaning == 'Prostate')]

In [ ]:
temp_prostate_qin_monai_df['time_index'] = temp_prostate_qin_monai_df.apply(calculate_time_index,
                                    args = (temp_prostate_qin_monai_df,), axis=1)
temp_prostate_qin_monai_df["PID"] = temp_prostate_qin_monai_df.apply(lambda x: "PID_"\
                                                               +x["image_PatientID"].split('-')[1]\
                             +"_"+x["time_index"], axis=1)

In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#ffbf00']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_monai_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : "        "+x.split("_")[0][-2:]).values)
x_ticks_formatted = [x if idx % 2 == 0 else "" for idx, x in enumerate(x_ticks_formatted)]
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai3 = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
            #  hover_data=["PatientID"],
              markers=True,
              width=1100, height=400)
# initializing showlegend to "False"
fig_ai3.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai3.update_xaxes(tickangle=0, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai3.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai3['data']):
  # if idx % 2 == 0 :
  trace["name"] = "monai-prostate158"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai3['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai3.show()

Whole prostate bamf_nnunet_mr_prostate

In [ ]:
temp_prostate_qin_bamf_prostate_df = temp_df[(temp_df.algorithmNameCollection == 'bamf_nnunet_mr_prostate-qin_prostate_repeatability') \
                                     & (temp_df.SegmentedPropertyTypeCodeMeaning == 'Prostatic structure')]

In [ ]:
temp_prostate_qin_bamf_prostate_df['time_index'] = temp_prostate_qin_bamf_prostate_df.apply(calculate_time_index,
                                    args = (temp_prostate_qin_bamf_prostate_df,), axis=1)
temp_prostate_qin_bamf_prostate_df["PID"] = temp_prostate_qin_bamf_prostate_df.apply(lambda x: "PID_"\
                                                               +x["image_PatientID"].split('-')[1]\
                             +"_"+x["time_index"], axis=1)

In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#40E0D0']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_bamf_prostate_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : "        "+x.split("_")[0][-2:]).values)
x_ticks_formatted = [x if idx % 2 == 0 else "" for idx, x in enumerate(x_ticks_formatted)]
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai4 = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
            #  hover_data=["PatientID"],
              markers=True,
              width=1100, height=400)
# initializing showlegend to "False"
fig_ai4.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai4.update_xaxes(tickangle=0, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai4.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai4['data']):
  # if idx % 2 == 0 :
  trace["name"] = "bamf_nnunet_mr_prostate"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai4['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai4.show()

QIN whole prostate expert figure

In [ ]:
###
temp_df_plot = temp_prostate_qin_task024_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
# scatter plot using plotly
#00FF0
pio.templates["custom_gt_idc"] = go.layout.Template(layout=go.Layout(colorway=['#00008b']))
pio.templates.default = 'custom_gt_idc'
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : "        "+x.split("_")[0][-2:]).values)
x_ticks_formatted = [x if idx % 2 == 0 else "" for idx, x in enumerate(x_ticks_formatted)]
temp_df_plot["Volume of Mesh"] = temp_df_plot.expert_Value.apply(lambda x : x/1000)#convert to mL
fig_gt_idc = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
            #  points="all",
             color="image_PatientID",
             hover_data=["image_PatientID"],
              markers=True,
              width=1100, height=400)

# initializing showlegend to "False"
fig_gt_idc.update_layout(showlegend=True, font=dict(size=12))
fig_gt_idc.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_gt_idc.update_xaxes(tickangle=0, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
for idx, trace in enumerate(fig_gt_idc['data']):
  # if idx % 2 == 0 :
  trace["name"] = "Expert"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_gt_idc['data']):
    if idx > 0: trace['showlegend'] = False
fig_gt_idc.show()

Combined whole prostate volume repeatability figure


In [ ]:
### Combined

#legend idc
fig = go.Figure(data = fig_gt_idc.data + fig_ai.data + fig_ai2.data + fig_ai3.data + fig_ai4.data)
fig.update_layout(showlegend=True)
fig.update_xaxes(tickangle=0, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig.update_layout(
                  width=1100, height=400,
                  title='QIN-Prostate-REPEATABILITY collection, repeatability study of volume between AI and expert whole prostate segmentations',
                  xaxis_title="CaseID",
                  yaxis_title="Volume, mL",
                  font=dict(size=10),
                  font_color="black")
fig.update_layout(margin=dict(l=80, r=80, t=80, b=80))
fig.update_layout(legend=dict(orientation = "v", yanchor="bottom",y=0.6,xanchor="right", x=1.2))
fig.show()

### Repeatability analysis for ProstateX

In [ ]:
data_prostatex_inf_only_df['time_index'] = data_prostatex_inf_only_df.apply(calculate_time_index_prostatex_inf_only,
                                    args = (data_prostatex_inf_only_df,), axis=1)

In [ ]:
data_prostatex_inf_only_df.time_index.unique()

array(['T0', 'T1', 'T2'], dtype=object)

In [ ]:
data_prostatex_inf_only_df["PID"] = data_prostatex_inf_only_df.apply(lambda x: "PID_"\
                                                               +x["image_PatientID"].split('-')[1]\
                             +"_"+x["time_index"], axis=1)

In [ ]:
len(sorted(data_prostatex_inf_only_df.PID.unique()))

419

In [ ]:
repeat_PIDs = [x.split("_")[1] for x in sorted(data_prostatex_inf_only_df.PID.unique()) if x.split("_")[-1]=="T1"]
# repeat_PIDs = [x for x in repeat_PIDs if x != "0031"]#exclude this case, since one model does not have segmentation
repeat_PIDs = [x for x in sorted(data_prostatex_inf_only_df.PID.unique()) if x.split("_")[1] in repeat_PIDs]

In [ ]:
plot_data_prostatex_inf_only_df = data_prostatex_inf_only_df[data_prostatex_inf_only_df.PID.isin(repeat_PIDs)]
plot_data_prostatex_inf_only_df = plot_data_prostatex_inf_only_df[plot_data_prostatex_inf_only_df.SegmentedPropertyTypeCodeMeaning.isin(["Prostatic structure",
                                                                                                                                        "Prostate"])]
plot_data_prostatex_inf_only_df = plot_data_prostatex_inf_only_df.reset_index(drop=True)

In [ ]:
plot_data_prostatex_inf_only_df.SegmentAlgorithmName.unique()

array(['bamf_nnunet_mr_prostate', 'nnunet_prostate_task24',
       'nnunet_prostate_zonal_task05', 'monai_prostate158'], dtype=object)

In [ ]:
plot_data_prostatex_inf_only_df.time_index.unique()

array(['T1', 'T0', 'T2'], dtype=object)

Cases to exlude -- make sure every method share the same set of cases.
Some method can have None values for some cases

In [ ]:
cases_exclude = "PID_0148|PID_0052|PID_0272|PID_0260"

bamf_nnunet_mr_prostate

In [ ]:
temp_prostate_qin_bamf_nnunet_mr_prostate_df = plot_data_prostatex_inf_only_df[(plot_data_prostatex_inf_only_df.SegmentAlgorithmName == 'bamf_nnunet_mr_prostate')]
temp_prostate_qin_bamf_nnunet_mr_prostate_df = add_missing_PID_df(plot_data_prostatex_inf_only_df, temp_prostate_qin_bamf_nnunet_mr_prostate_df)

In [ ]:
len(temp_prostate_qin_bamf_nnunet_mr_prostate_df.PID.unique())

154

In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#8b0000']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_bamf_nnunet_mr_prostate_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
# x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : ""+x.split("_")[0][-2:]).values)
# x_ticks_formatted = [x if idx % 2 == 0 else "" for idx, x in enumerate(x_ticks_formatted)]
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : int(x.split("_")[0])).values)
x_ticks_formatted_updated = []
for idx, x in enumerate(x_ticks_formatted):
  if idx == 0:
    x_ticks_formatted_updated.append(x)
  else:
    if idx<len(x_ticks_formatted)-1 and idx>0:
      if x_ticks_formatted[idx+1] == x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      elif x_ticks_formatted[idx+1] != x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      else:
        x_ticks_formatted_updated.append("")
    else:
      x_ticks_formatted_updated.append("")
x_ticks_formatted = x_ticks_formatted_updated
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai_bamf = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
             hover_data=["time_index"],
              markers=True,
              width=2200, height=400)
# initializing showlegend to "False"
fig_ai_bamf.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai_bamf.update_xaxes(tickangle=80, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai_bamf.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai_bamf['data']):
  # if idx % 2 == 0 :
  trace["name"] = "bamf_nnunet_mr_prostate"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai_bamf['data']):
    if idx > 0: trace['showlegend'] = False
fig_ai_bamf.show()

nnunet_prostate_task24

In [ ]:
temp_prostate_qin_nnunet_prostate_task24_df = plot_data_prostatex_inf_only_df[plot_data_prostatex_inf_only_df.SegmentAlgorithmName == 'nnunet_prostate_task24']
temp_prostate_qin_nnunet_prostate_task24_df = add_missing_PID_df(plot_data_prostatex_inf_only_df, temp_prostate_qin_nnunet_prostate_task24_df)

In [ ]:
len(temp_prostate_qin_nnunet_prostate_task24_df.PID.unique())

154

In [ ]:
print(list(set(temp_prostate_qin_nnunet_prostate_task24_df.PID.unique()) - set(temp_prostate_qin_bamf_nnunet_mr_prostate_df.PID.unique())))
print(list(set(temp_prostate_qin_bamf_nnunet_mr_prostate_df.PID.unique()) - set(temp_prostate_qin_nnunet_prostate_task24_df.PID.unique())))

[]
[]


In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#0b6623']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_nnunet_prostate_task24_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : int(x.split("_")[0])).values)
x_ticks_formatted_updated = []
for idx, x in enumerate(x_ticks_formatted):
  if idx == 0:
    x_ticks_formatted_updated.append(x)
  else:
    if idx<len(x_ticks_formatted)-1 and idx>0:
      if x_ticks_formatted[idx+1] == x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      elif x_ticks_formatted[idx+1] != x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      else:
        x_ticks_formatted_updated.append("")
    else:
      x_ticks_formatted_updated.append("")
x_ticks_formatted = x_ticks_formatted_updated
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai_nnunet_024 = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
            #  hover_data=["PatientID"],
              markers=True,
              width=2200, height=400)
# initializing showlegend to "False"
fig_ai_nnunet_024.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai_nnunet_024.update_xaxes(tickangle=80, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai_nnunet_024.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai_nnunet_024['data']):
  # if idx % 2 == 0 :
  trace["name"] = "nnunet_prostate_task24"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai_nnunet_024['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai_nnunet_024.show()

monai_prostate158

In [ ]:
temp_prostate_qin_monai_prostate158_df = plot_data_prostatex_inf_only_df[plot_data_prostatex_inf_only_df.SegmentAlgorithmName == 'monai_prostate158']
temp_prostate_qin_monai_prostate158_df = add_missing_PID_df(plot_data_prostatex_inf_only_df, temp_prostate_qin_monai_prostate158_df)

In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#ffbf00']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_monai_prostate158_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : int(x.split("_")[0])).values)
x_ticks_formatted_updated = []
for idx, x in enumerate(x_ticks_formatted):
  if idx == 0:
    x_ticks_formatted_updated.append(x)
  else:
    if idx<len(x_ticks_formatted)-1 and idx>0:
      if x_ticks_formatted[idx+1] == x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      elif x_ticks_formatted[idx+1] != x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      else:
        x_ticks_formatted_updated.append("")
    else:
      x_ticks_formatted_updated.append("")
x_ticks_formatted = x_ticks_formatted_updated
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai_monai_prostate158 = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
            #  hover_data=["PatientID"],
              markers=True,
              width=2200, height=400)
# initializing showlegend to "False"
fig_ai_monai_prostate158.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai_monai_prostate158.update_xaxes(tickangle=80, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai_monai_prostate158.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai_monai_prostate158['data']):
  # if idx % 2 == 0 :
  trace["name"] = "monai_prostate158"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai_monai_prostate158['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai_monai_prostate158.show()

nnunet_prostate_zonal_task05

In [ ]:
temp_prostate_qin_nnunet_prostate_zonal_task05_df = plot_data_prostatex_inf_only_df[plot_data_prostatex_inf_only_df.SegmentAlgorithmName == 'nnunet_prostate_zonal_task05']
temp_prostate_qin_nnunet_prostate_zonal_task05_df= add_missing_PID_df(plot_data_prostatex_inf_only_df, temp_prostate_qin_nnunet_prostate_zonal_task05_df)

In [ ]:
len(temp_prostate_qin_nnunet_prostate_zonal_task05_df.PID.unique())

154

In [ ]:
print(list(set(temp_prostate_qin_monai_prostate158_df.PID.unique()) - set(temp_prostate_qin_nnunet_prostate_zonal_task05_df.PID.unique())))

[]


In [ ]:
# scatter plot using plotly
pio.templates["custom_nnUNet"] = go.layout.Template(layout=go.Layout(colorway=['#40E0D0']))
pio.templates.default = 'custom_nnUNet'
temp_df_plot = temp_prostate_qin_nnunet_prostate_zonal_task05_df.sort_values(by="PID").copy()
temp_df_plot["PID"] = temp_df_plot["PID"].apply(lambda x : "_".join(x.split("_")[1:]))
x_ticks_formatted = list(temp_df_plot["PID"].apply(lambda x : int(x.split("_")[0])).values)
x_ticks_formatted_updated = []
for idx, x in enumerate(x_ticks_formatted):
  if idx == 0:
    x_ticks_formatted_updated.append(x)
  else:
    if idx<len(x_ticks_formatted)-1 and idx>0:
      if x_ticks_formatted[idx+1] == x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      elif x_ticks_formatted[idx+1] != x and x_ticks_formatted[idx-1] != x:
        x_ticks_formatted_updated.append(x)
      else:
        x_ticks_formatted_updated.append("")
    else:
      x_ticks_formatted_updated.append("")
x_ticks_formatted = x_ticks_formatted_updated
temp_df_plot["Volume of Mesh"] = temp_df_plot.ai_Value.apply(lambda x : x/1000)#convert to mL
# temp_df['PID'] = ['\n'.join(x.split("_")) for x in  temp_df['PID']]
fig_ai_nnunet_05 = px.line(temp_df_plot,
             x = "PID",
             y="Volume of Mesh",
             color="image_PatientID",
            #  hover_data=["PatientID"],
              markers=True,
              width=2200, height=400)
# initializing showlegend to "False"
fig_ai_nnunet_05.update_layout(showlegend=True, font=dict(size=10), legend_title="",
                     legend=dict(
                      font=dict(size= 9)))
fig_ai_nnunet_05.update_xaxes(tickangle=80, ticktext=x_ticks_formatted,
                        tickvals=list(temp_df_plot["PID"].values))
fig_ai_nnunet_05.update_traces(showlegend=True)
for idx, trace in enumerate(fig_ai_nnunet_05['data']):
  # if idx % 2 == 0 :
  trace["name"] = "nnunet_prostate_zonal_task05"
  # else:
    # trace["name"] = "AI segmentations"
# set showlegend property by name of trace
for idx, trace in enumerate(fig_ai_nnunet_05['data']):
    if idx > 0: trace['showlegend'] = False

fig_ai_nnunet_05.show()

In [ ]:
### Combined
#legend idc
fig = go.Figure(data = fig_ai_bamf.data + fig_ai_nnunet_024.data + fig_ai_monai_prostate158.data + fig_ai_nnunet_05.data)
fig.update_layout(showlegend=True)
fig.update_xaxes(tickangle=80, ticktext=x_ticks_formatted,
                        tickvals=temp_df_plot["PID"].values)
fig.update_layout(
                  width=1500, height=300,
                  title='ProstateX collection, repeatability study of AI volumes between same study different T2s for whole prostate segmentations',
                  xaxis_title="CaseID",
                  yaxis_title="Volume, mL",
                  font=dict(size=13),
                  font_color="black")
fig.update_layout(margin=dict(l=80, r=80, t=80, b=80))
fig.update_layout(legend=dict(orientation = "v", yanchor="bottom",y=0.6,xanchor="right", x=1.15))
fig.show()